In [10]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import numpy as np
from scipy.optimize import minimize

# The Hamiltonian as SparsePauliOp
from qiskit.quantum_info import SparsePauliOp

# The Hamiltonian as SparsePauliOp
ham = SparsePauliOp.from_list([('IIZZ', -0.5 + 0.j), 
                               ('IZIZ', -0.5 + 0.j), 
                               ('IZZI', -0.5 + 0.j), 
                               ('ZIIZ', -0.5 + 0.j), 
                               ('ZZII', -0.5 + 0.j)])

# Convert the Hamiltonian to a list of Pauli objects
def pauli_string_converter(ham):
    pauli_objects = []
    for term, coeff in ham.to_list():
        pauli_objects.append((coeff, term))  # Pauli term as a string
    return pauli_objects

# Translate the Hamiltonian
pauli_objects = pauli_string_converter(ham)

# Display the Pauli terms and their coefficients
for obj in pauli_objects:
    print(f"Weight: {obj[0]}, PauliString: {obj[1]}")

# Function to create the quantum circuit with a given number of qubits and layers
def create_circuit(n_qubits, n_layers, theta):
    qc = QuantumCircuit(n_qubits)
    
    # Apply Hadamard gates to all qubits
    qc.h(range(n_qubits))

    for layer in range(n_layers):
        # Apply RX gates for each layer with specific parameters
        for i in range(n_qubits):
            qc.rx(theta[layer * n_qubits + i], i)
        
        # Apply CNOT gates for each layer
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)

    return qc

# Function that applies the correct rotations to measure in the correct basis
def apply_meas_rotation(qc, pauli_string):
    for qubit, pauli in enumerate(pauli_string):
        if pauli == 'X':
            qc.h(qubit)  # Hadamard to measure in X
        elif pauli == 'Y':
            qc.sdg(qubit)  # SDG gate
            qc.h(qubit)    # Hadamard to measure in Y

# Function to calculate the expectation value of the Hamiltonian
def cost_function(circuit, pauli_objects, n_shots):
    backend = AerSimulator()
    cost_fun = 0
    for coeff, pauli_string in pauli_objects:
        qc_copy = circuit.copy()  # Copy the circuit to avoid modifications

        # Apply the correct rotations to measure in the correct basis
        apply_meas_rotation(qc_copy, pauli_string)

        # Measure all qubits in the computational basis
        qc_copy.measure_all()

        # Execute the circuit
        job = backend.run(transpile(qc_copy, backend), shots=n_shots)
        result = job.result()

        # Get the counts (measurement outcomes)
        counts = result.get_counts()

        # Calculate the expectation value
        exp_value = 0
        for outcome, count in counts.items():
            exp_value += coeff * (1 if outcome.count('1') % 2 == 0 else -1) * count

        cost_fun += exp_value / n_shots

    return np.real(cost_fun)

# Wrapper function for the optimizer
def cost_function_opt(theta, n_qubits, n_layers, pauli_objects, n_shots):
    # Create the quantum circuit for optimization
    qc = create_circuit(n_qubits, n_layers, theta)
    return cost_function(qc, pauli_objects, n_shots)

# Initial random parameters
n_qubits = 4
n_layers = 3  # Number of layers in the circuit
theta_init = np.random.uniform(0, 2 * np.pi, size=n_layers * n_qubits)

# Run optimization using BFGS
result = minimize(cost_function_opt, theta_init, args=(n_qubits, n_layers, pauli_objects, 1024), method='BFGS')

# Extract optimal parameters and cost
optimal_theta = result.x
optimal_cost = result.fun

print("Optimal parameters:", optimal_theta)
print("Minimum cost function value:", optimal_cost)


Weight: (-0.5+0j), PauliString: IIZZ
Weight: (-0.5+0j), PauliString: IZIZ
Weight: (-0.5+0j), PauliString: IZZI
Weight: (-0.5+0j), PauliString: ZIIZ
Weight: (-0.5+0j), PauliString: ZZII
Optimal parameters: [0.41243746 3.41661754 4.99461278 1.7333162  2.3990186  6.23836833
 6.10716814 1.28478328 6.12814185 3.27974835 5.32640125 1.11097501]
Minimum cost function value: -0.017578125
